In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.transforms import ToTensor
from pandas import Series
import numpy as np
from sklearn import preprocessing
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## get node 33,36,37,38,39

In [ ]:
# DATA = pd.read_parquet('/content/drive/My Drive/MasterProject/create_data_my_model_seprate_anomaly_0.parquet')
# DATA = pd.read_parquet('/content/drive/My Drive/MasterProject/create_data_my_model_add_label_2_lr_0.1.parquet')
# DATA = pd.read_parquet('/content/drive/My Drive/MasterProject/create_data_my_model_add_label_1_lr_0.03.parquet')
DATA = pd.read_parquet('/content/drive/My Drive/MasterProject/create_data_my_model_add_label_0.parquet')
DATA.head(5)

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value,next_value,timestamp
0,22.998926,0.087301,22.876736,23.163902,30.879311,0.150768,30.710751,31.134846,31.356607,0.132538,...,0.007004,11.888273,11.911995,711.028687,96.578575,560.916748,941.764587,0.022346,0.024368,2021-04-02 00:45:00+00:00
1,22.999556,0.086445,22.879534,23.164682,30.883602,0.150250,30.715548,31.132477,31.358051,0.132548,...,0.010639,11.888582,11.912056,711.218628,96.630974,560.983154,941.733704,0.022063,0.024217,2021-04-02 01:00:00+00:00
2,22.998312,0.088277,22.879469,23.164612,30.878307,0.150297,30.713680,31.133499,31.357065,0.132364,...,0.009165,11.888429,11.912006,711.016724,96.578659,560.909363,941.574463,0.023345,0.026302,2021-04-02 01:15:00+00:00
3,22.997112,0.088988,22.878490,23.163433,30.876856,0.150252,30.712833,31.131662,31.355062,0.132358,...,0.009096,11.888359,11.911938,711.126526,96.556007,561.039673,941.559937,0.023303,0.026262,2021-04-02 01:30:00+00:00
4,22.995653,0.088909,22.877846,23.162666,30.877129,0.150370,30.712425,31.132263,31.355190,0.132298,...,0.008816,11.888303,11.911971,711.149658,96.573868,561.017029,941.627258,0.023309,0.026246,2021-04-02 01:45:00+00:00


In [ ]:
DATA[DATA["value"]>0.0331]

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value,next_value,timestamp
1423,22.823292,0.093859,22.670609,22.913786,30.653034,0.147105,30.437914,30.862164,31.126539,0.124720,...,0.005372,11.874552,11.900568,702.216797,95.904015,556.335876,934.670715,0.033333,0.035104,2021-04-16 20:30:00+00:00
1424,22.825979,0.093246,22.671040,22.918810,30.654831,0.147092,30.441347,30.866323,31.130352,0.125538,...,0.004687,11.874817,11.900764,702.373474,95.917854,556.378784,934.855835,0.033408,0.035089,2021-04-16 20:45:00+00:00
1425,22.827133,0.092105,22.674547,22.918900,30.656168,0.146781,30.446140,30.867189,31.133188,0.125523,...,0.007371,11.875104,11.900821,702.564819,95.954086,556.434998,934.856750,0.033193,0.035040,2021-04-16 21:00:00+00:00
1426,22.829542,0.091813,22.674940,22.919798,30.659176,0.146876,30.445709,30.866331,31.132147,0.125248,...,0.007541,11.875127,11.900910,702.776672,95.954269,556.647583,934.945618,0.033104,0.034896,2021-04-16 21:15:00+00:00
1534,22.846260,0.091358,22.686600,22.924303,30.630341,0.148763,30.407249,30.835688,31.095472,0.122952,...,0.009009,11.873402,11.899488,702.204712,95.815903,555.888306,933.573914,0.033196,0.035511,2021-04-18 00:15:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23153,22.838264,0.092542,22.710186,23.024261,30.615204,0.145955,30.442579,30.854660,31.099174,0.140402,...,0.006600,11.882055,11.906571,700.943115,94.553017,554.082947,929.262634,0.033147,0.032140,2021-11-29 05:00:00+00:00
23154,22.840858,0.092441,22.711866,23.025948,30.618036,0.145868,30.443510,30.854410,31.098093,0.140308,...,0.006171,11.882091,11.906693,701.106506,94.544174,554.247681,929.345764,0.033133,0.032020,2021-11-29 05:15:00+00:00
23157,22.840654,0.092903,22.709850,23.027012,30.619812,0.146256,30.443714,30.859383,31.103609,0.140215,...,0.003290,11.881963,11.906795,700.908386,94.519661,554.285278,929.617065,0.033166,0.032016,2021-11-29 06:00:00+00:00
23434,22.909321,0.085072,22.768679,23.108984,30.689281,0.147488,30.511616,30.941639,31.189631,0.143242,...,0.004222,11.887554,11.911137,705.834839,95.002716,556.683960,934.273682,0.033186,0.031227,2021-12-02 03:15:00+00:00


In [ ]:
## chang data -create_data_my_model_add_label_2_lr_0.1
# DATA.loc[DATA['value'] > 0.08, 'value'] = 1
## chang data -create_data_my_model_add_label_2_lr_0.03
# DATA.loc[DATA['value'] > 0.0358, 'value'] = 1
## chang data -create_data_my_model_add_label_0_lr_0.01
DATA.loc[DATA['value'] > 0.0331, 'value'] = 1

DATA = DATA.drop(columns=['next_value'])

In [ ]:
#The dataset has periods of unavailability of Nagios traces, as described in the dataset paper:  https://www.nature.com/articles/s41597-023-02174-3/figures/3
#We suggest dropping the periods where the labels are largely unavailable and using the dataset either after 1.4.2021 or 1.10.2021.
DATA = DATA[DATA['timestamp'] > '2021-04-01']

### parisa change
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)
## parisa change

In [ ]:
DATA.reset_index(drop=True, inplace = True)
DATA = DATA.fillna(0)
DATA['timestamp'] = pd.to_datetime(DATA['timestamp'])
l = DATA.timestamp.diff() == pd.Timedelta(minutes=15) #time consistency -> measurements should be 15 minutes apart
chunks = []
current_chunk = []
for index, value in enumerate(l):
    current_chunk.append(DATA.iloc[index])
    if not value:
        chunks.append(pd.DataFrame(current_chunk))
        current_chunk = []


if current_chunk:
    chunks.append(pd.DataFrame(current_chunk))

In [ ]:
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change
relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
revelent_len = [len(c) for c in chunks if len(c) >= 20]
DATA2 = pd.concat(relevant)
DATA = DATA2
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change end
##3 parisa change
# relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
# DATA2 = pd.concat(relevant)
# DATA = DATA2
# DATA['value'] = DATA['value'].replace(2,1)
# DATA['value'] = DATA['value'].replace(3,1)
# DATA = DATA.drop(columns=['timestamp'])
# DATA = DATA.astype(float)
##3 parisa change end

In [ ]:
DATA3= DATA
DATA=DATA.drop(columns=['timestamp'])
DATA = DATA.astype(float)

In [ ]:
scaler = preprocessing.MinMaxScaler()
names = DATA.columns
d = scaler.fit_transform(DATA)
DATA = pd.DataFrame(d, columns=names)
### parisa change
timestamp_scaler = preprocessing.MinMaxScaler()
DATA['timestamp'] = timestamp_scaler.fit_transform(DATA3['timestamp'].astype('int64').values.reshape(-1, 1))
#### parisa change end

In [ ]:
### to get rid of time and test
# DATA=DATA.drop(columns=['timestamp'])

In [ ]:
train_data = DATA[:int(DATA.shape[0]*0.8)]
test_data = DATA[int(DATA.shape[0]*0.8):]

In [ ]:
train_data.shape,test_data.shape

((41830, 354), (10458, 354))

In [ ]:
train_data.shape

(41847, 1382)

In [ ]:
train_data['value'].value_counts()

,count
value,
0.0,40673
1.0,1174


In [ ]:
train_feat = train_data.drop(columns=['value'])
train_feat = train_feat.to_numpy()
test_feat = test_data.drop(columns=['value'])
test_feat = test_feat.to_numpy()

In [ ]:
train_feat = torch.tensor(train_feat, dtype=torch.float)
test_feat = torch.tensor(test_feat, dtype=torch.float)

In [ ]:
test_labels = test_data['value']
test_labels = test_labels.to_numpy()
test_labels = torch.tensor(test_labels, dtype=torch.float)
test_labels

tensor([0.0096, 0.0095, 0.0096,  ..., 0.0093, 0.0094, 0.0094])

In [ ]:
train_window = 20 ## 20

def create_data_seq(data,tw):
    seq = []
    for i in range(len(data)-tw):
        x_seq = data[i:i+tw],
        y_seq = data[i+tw:i+tw+1]  ###در ترین میاد 0 تا 20 رو میده و میگه 21 رو حدس بزن
        seq.append((x_seq,y_seq))
    return seq


In [ ]:
# train_seq = create_data_seq(train_feat,train_window,revelent_len) ## parisa change
train_seq = create_data_seq(train_feat,train_window) ## parisa change

In [ ]:
X[0].size()


torch.Size([1, 20, 353])

In [ ]:
train_loader = DataLoader(train_seq, batch_size=1, shuffle=True)

X,y  = next(iter(train_loader))
# print(X)


In [ ]:
# test_seq = create_data_seq(test_feat,train_window,revelent_len)
test_seq = create_data_seq(test_feat,train_window)

In [ ]:
test_loader = DataLoader(test_seq, batch_size=1, shuffle=False) ## در هر فور ما یه تعدادی بچ داریم که میدیم به دل با بچ بیشتر میتونیم بگیم مثلا 2 تا 2 تا بهش بده
X,y = next(iter(test_loader))

In [ ]:
class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = torch.nn.LSTM(
            input_size=1382,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16,
          hidden_size=8,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x[:, :, :]

In [ ]:
############### mine
# class encoder(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.lstm1 = torch.nn.LSTM(
#             input_size=352,
#             hidden_size=32,
#             batch_first=True,
#             num_layers=1
#         )
#         self.lstm2 = torch.nn.LSTM(
#           input_size=32,
#           hidden_size=8,
#           num_layers=1,
#           batch_first=True
#         )

#     def forward(self, x):
#         batch_size = x.shape[0]
#         h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)

#         x, (_, _) = self.lstm1(x, (h0, c0))

#         h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
#         x, (_, _) = self.lstm2(x, (h0, c0))
#         return x[:, :, :]

class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer
        self.lstm1 = torch.nn.LSTM(
            input_size=1381,
            hidden_size=32,  # Increased the hidden size for this example
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=32,   # Input size matches the hidden size of lstm1
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer (newly added)
        self.lstm3 = torch.nn.LSTM(
            input_size=16,   # Input size matches the hidden size of lstm2
            hidden_size=8,
            batch_first=True,
            num_layers=1
        )

    def forward(self, x):
        batch_size = x.shape[0]

        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        x, (_, _) = self.lstm1(x, (h0, c0))

        # Initial hidden and cell states for lstm2
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))

        # Initial hidden and cell states for lstm3
        h0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        x, (_, _) = self.lstm3(x, (h0, c0))

        return x[:, :, :]

In [ ]:
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16,
          hidden_size=1381,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size, 1381).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 1381).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x

In [ ]:
### mine
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer (added for symmetry with the encoder)
        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=16,
            hidden_size=32,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer
        self.lstm3 = torch.nn.LSTM(
            input_size=32,
            hidden_size=1381,
            batch_first=True,
            num_layers=1
        )

    def forward(self, x):
        batch_size = x.shape[0]

        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        x, (_, _) = self.lstm1(x, (h0, c0))

        # Initial hidden and cell states for lstm2
        h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))

        # Initial hidden and cell states for lstm3
        h0 = torch.zeros(1, batch_size, 1381).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 1381).requires_grad_().to(device)
        x, (_, _) = self.lstm3(x, (h0, c0))

        return x

In [ ]:
############### mine for add h,c

class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer
        self.lstm1 = torch.nn.LSTM(
            input_size=353 ,
            hidden_size=32,  # Increased the hidden size for this example
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=32,   # Input size matches the hidden size of lstm1
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer (newly added)
        self.lstm3 = torch.nn.LSTM(
            input_size=16,   # Input size matches the hidden size of lstm2
            hidden_size=8,
            batch_first=True,
            num_layers=1
        )
        self.h_proj_1_to_2 = torch.nn.Linear(32, 16)
        self.c_proj_1_to_2 = torch.nn.Linear(32, 16)

        self.h_proj_2_to_3 = torch.nn.Linear(16, 8)
        self.c_proj_2_to_3 = torch.nn.Linear(16, 8)

    def forward(self, x):
        batch_size = x.shape[0]
        # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))

        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 32).to(x.device)
        c0 = torch.zeros(1, batch_size, 32).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h0, c0))

        # Resize h1, c1 to match the input size of lstm2
        h1 = self.h_proj_1_to_2(h1)
        c1 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h1, c1))

        # Resize h2, c2 to match the input size of lstm3
        h2 = self.h_proj_2_to_3(h2)
        c2 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h2, c2))
        return x[:, :, :]

In [ ]:
### mine for add h,c
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer (added for symmetry with the encoder)
        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=16,
            hidden_size=32,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer
        self.lstm3 = torch.nn.LSTM(
            input_size=32,
            hidden_size=353,
            batch_first=True,
            num_layers=1
        )

        self.h_proj_1_to_2 = torch.nn.Linear(16, 32)
        self.c_proj_1_to_2 = torch.nn.Linear(16, 32)

        self.h_proj_2_to_3 = torch.nn.Linear(32, 353)
        self.c_proj_2_to_3 = torch.nn.Linear(32, 353)

    def forward(self, x):
        batch_size = x.shape[0]

        # # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))


        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        h1 = torch.zeros(1, batch_size, 16).to(x.device)
        c1 = torch.zeros(1, batch_size, 16).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h1, c1))

        # Resize h1, c1 to match the input size of lstm2
        h2 = self.h_proj_1_to_2(h1)
        c2 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h2, c2))

        # Resize h2, c2 to match the input size of lstm3
        h3 = self.h_proj_2_to_3(h2)
        c3 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h3, c3))

        return x

In [ ]:
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.decoder = decoder().to(device)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8,16)
        self.fc2 = torch.nn.Linear(16,353)


    def forward(self, x):
        x = self.encoder(x)
        #print('LSTM out', x.shape)
        x = self.fc1(x)
        #print('DENSE 1', x.shape)
        x = self.fc2(x)
        #print('DENSE 2', x.shape)

        return x

In [ ]:
#### mine
# class RUAD(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.encoder = encoder().to(device)
#         self.fc1 = torch.nn.Linear(8,16)
#         self.fc2 = torch.nn.Linear(16,352)


#     def forward(self, x):
#         x = self.encoder(x)
#         #print('LSTM out', x.shape)
#         x = self.fc1(x)
#         #print('DENSE 1', x.shape)
#         x = self.fc2(x)
#         #print('DENSE 2', x.shape)

#         return x

In [ ]:
### mine
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8, 16)
        self.fc2 = torch.nn.Linear(16, 8)
        self.decoder = decoder().to(device)  # Adding the decoder to RUAD

    def forward(self, x):
        x = self.encoder(x)
        # Passing through the fully connected layers   ----- new changes
        # x = self.fc1(x)
        # x = self.fc2(x)
        # Passing the result through the decoder ------- end of new changes
        x = self.decoder(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = RUAD()
model

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(353, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 353, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=353, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=353,

In [ ]:
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  ### adam ro bezar va bebin
criterion = torch.nn.MSELoss()

In [ ]:
for d in train_loader:
    print(len(d[0]))
    print(d[0][0].size())
    print(d[1].size())
    break
    ## batch size,(row count,column count) --> sequence

1
torch.Size([1, 20, 353])
torch.Size([1, 1, 353])


In [ ]:
print(model)

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(353, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 353, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=353, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=353,

In [ ]:
%%time
def train():
    model.train()
    for d in train_loader:
        #print(type(d))
        x = d[0][0].to(device)
        y = d[1].to(device)
        optimizer.zero_grad()
        out = model(x)
        # print(out[0][9])
        loss = criterion(out[0][train_window-1], y[0][0])
        loss.backward()
        optimizer.step()
    return float(loss)

for epoch in range(6):
    loss = train()
    print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')




############ results
########### result change layer 8 to 32
# Epoch: 01, Loss: 0.0188
# Epoch: 02, Loss: 0.0273
# Epoch: 03, Loss: 0.0033
# CPU times: user 6min 3s, sys: 3.62 s, total: 6min 6s
# Wall time: 6min 12s

########### result add hidden ayer 32
# Epoch: 01, Loss: 0.0021
# Epoch: 02, Loss: 0.0378
# Epoch: 03, Loss: 0.0069
# CPU times: user 7min 58s, sys: 4.28 s, total: 8min 2s
# Wall time: 8min 11s

########### result add hidden layer 32 to decoder too and add decoder to ruad model with gpu
# Epoch: 01, Loss: 0.0200
# Epoch: 02, Loss: 0.0210
# Epoch: 03, Loss: 0.0231
# CPU times: user 13min 7s, sys: 14.9 s, total: 13min 22s
# Wall time: 13min 5s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu
# Epoch: 01, Loss: 0.0089
# Epoch: 02, Loss: 0.0483
# Epoch: 03, Loss: 0.0035
# CPU times: user 12min 38s, sys: 13.5 s, total: 12min 51s
# Wall time: 12min 34s

########### result first ruad model with gpu and add timestamp column and window 10 and seq
# Epoch: 01, Loss: 0.0107
# Epoch: 02, Loss: 0.0098
# Epoch: 03, Loss: 0.0219
# CPU times: user 11min 17s, sys: 4.6 s, total: 11min 22s
# Wall time: 11min 28s

########### result first ruad model with gpu and add timestamp column and window 10 and remove seq
# Epoch: 01, Loss: 0.0252
# Epoch: 02, Loss: 0.0093
# Epoch: 03, Loss: 0.0188
# CPU times: user 11min 8s, sys: 4.67 s, total: 11min 12s
# Wall time: 11min 19s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , adam optimizer
# Epoch: 01, Loss: 0.0365
# Epoch: 02, Loss: 0.0473
# Epoch: 03, Loss: 0.0536
# CPU times: user 12min 14s, sys: 13.8 s, total: 12min 28s
# Wall time: 12min 12s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer
# Epoch: 01, Loss: 0.0594
# Epoch: 02, Loss: 0.0260
# Epoch: 03, Loss: 0.0311
# CPU times: user 11min 27s, sys: 13.3 s, total: 11min 40s
# Wall time: 11min 24s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 8
# Epoch: 01, Loss: 0.0270
# Epoch: 02, Loss: 0.0267
# Epoch: 03, Loss: 0.0317
# Epoch: 04, Loss: 0.0313
# Epoch: 05, Loss: 0.0615
# CPU times: user 19min 21s, sys: 23.7 s, total: 19min 44s
# Wall time: 19min 18s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 5 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# CPU times: user 20min 12s, sys: 12 s, total: 20min 24s
# Wall time: 20min 39s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# Epoch: 06, Loss: 0.0030
# CPU times: user 24min 16s, sys: 14.37 s, total: 24min 30s
# Wall time: 24min 48s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0042
# Epoch: 02, Loss: 0.0035
# Epoch: 03, Loss: 0.0061
# Epoch: 04, Loss: 0.0053
# Epoch: 05, Loss: 0.0214
# Epoch: 06, Loss: 0.0109
# CPU times: user 26min 13s, sys: 14.2 s, total: 26min 27s
# Wall time: 26min 46s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1351
# Epoch: 01, Loss: 0.0111
# Epoch: 02, Loss: 0.2479
# Epoch: 03, Loss: 0.0097
# Epoch: 04, Loss: 0.0052
# Epoch: 05, Loss: 0.0080
# Epoch: 06, Loss: 0.0088
# CPU times: user 41min 57s, sys: 15.8 s, total: 42min 12s
# Wall time: 42min 38s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1351
# Epoch: 01, Loss: 0.0080
# Epoch: 02, Loss: 0.0062
# Epoch: 03, Loss: 0.0100
# Epoch: 04, Loss: 0.2012
# Epoch: 05, Loss: 0.0097
# Epoch: 06, Loss: 0.0084
# CPU times: user 47min 20s, sys: 21.4 s, total: 47min 41s
# Wall time: 48min 17s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6  ,len(data)=1351
# Epoch: 01, Loss: 0.0100
# Epoch: 02, Loss: 0.0079
# Epoch: 03, Loss: 0.0118
# Epoch: 04, Loss: 0.0127
# Epoch: 05, Loss: 0.2008
# Epoch: 06, Loss: 0.1996
# CPU times: user 1h 1min 51s, sys: 48.3 s, total: 1h 2min 40s
# Wall time: 1h 1min 51s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1350 (remove time)
# Epoch: 01, Loss: 0.0132
# Epoch: 02, Loss: 0.0073
# Epoch: 03, Loss: 0.0063
# Epoch: 04, Loss: 0.0073
# Epoch: 05, Loss: 0.0050
# Epoch: 06, Loss: 0.0042
# CPU times: user 40min 43s, sys: 12.6 s, total: 40min 56s
# Wall time: 41min 23s

###### new ones

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=353 (time) - create_data_my_model_seprate_anomaly_0
# Epoch: 01, Loss: 0.0014
# Epoch: 02, Loss: 0.0011
# Epoch: 03, Loss: 0.0006
# Epoch: 04, Loss: 0.0012
# Epoch: 05, Loss: 0.0007
# Epoch: 06, Loss: 0.0010
# CPU times: user 15min 7s, sys: 4.22 s, total: 15min 11s
# Wall time: 15min 20s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=353 (time) - create_data_my_model_add_label_2_lr_0.1
# Epoch: 01, Loss: 0.0023
# Epoch: 02, Loss: 0.0253
# Epoch: 03, Loss: 0.0222
# Epoch: 04, Loss: 0.0114
# Epoch: 05, Loss: 0.0333
# Epoch: 06, Loss: 0.0069
# CPU times: user 30min 32s, sys: 8.7 s, total: 30min 41s
# Wall time: 30min 58s


########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=353 (time) - create_data_my_model_add_label_2_lr_0.03
# Epoch: 01, Loss: 0.0079
# Epoch: 02, Loss: 0.0141
# Epoch: 03, Loss: 0.0093
# Epoch: 04, Loss: 0.0053
# Epoch: 05, Loss: 0.0083
# Epoch: 06, Loss: 0.0143
# CPU times: user 30min 45s, sys: 8.48 s, total: 30min 53s
# Wall time: 31min 11s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=353 (time) - create_data_my_model_add_label_2_lr_0.01
# Epoch: 01, Loss: 0.0111
# Epoch: 02, Loss: 0.0142
# Epoch: 03, Loss: 0.0372
# Epoch: 04, Loss: 0.0314
# Epoch: 05, Loss: 0.0337
# Epoch: 06, Loss: 0.0129
# CPU times: user 30min 33s, sys: 8.56 s, total: 30min 41s
# Wall time: 30min 58s

Epoch: 01, Loss: 0.0111
Epoch: 02, Loss: 0.0142
Epoch: 03, Loss: 0.0372
Epoch: 04, Loss: 0.0314
Epoch: 05, Loss: 0.0337
Epoch: 06, Loss: 0.0129
CPU times: user 30min 33s, sys: 8.56 s, total: 30min 41s
Wall time: 30min 58s


In [ ]:
def test(x):
    model.eval()
    x = x.to(device)
    out = model(x)
    return out

In [ ]:
X, y = next(iter(test_loader))
#print(X.shape)
print(len(X[0][0]))

10


In [ ]:
out = test(X[0])

In [ ]:
for d in test_loader:
    print(d[0][0].size())
    break

torch.Size([1, 10, 353])


In [ ]:
for d in test_loader:
    x = d[0][0].to(device)
    y = d[1].to(device)
    out = model(x)
    break

In [ ]:
from sklearn.metrics import mean_squared_error,roc_auc_score
reconstruction = []
loss = []
#y_true = []
for d in test_loader:
    x = d[0][0]
    y = d[1]
    out = test(x)
    reconstruction.append(out)
    loss.append(mean_squared_error(out[0][train_window-1].detach().cpu().numpy(),y[0][0]))
    #y_true.append(d[1].detach().cpu().numpy())

In [ ]:
# test_seq_label = create_data_seq(test_labels,train_window,revelent_len)
test_seq_label = create_data_seq(test_labels,train_window)

In [ ]:
temp_list = []
for i in range(len(test_seq_label)):
    temp = test_seq_label[i][1].type(torch.int64)
    temp = temp.detach().cpu().numpy().tolist()
    temp_list.append(temp)

In [ ]:
y_true = []
for i in range(len(temp_list)):
    y_true.append(temp_list[i][0])

In [ ]:
error_df = pd.DataFrame({'loss': loss,
                        'true_class': y_true})
error_df.describe()

,loss,true_class
count,10438.000000,10438.0
mean,0.029010,0.0
std,0.018435,0.0
min,0.007756,0.0
25%,0.015071,0.0
50%,0.023292,0.0
75%,0.036888,0.0
max,0.122304,0.0


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(error_df['true_class'],error_df['loss'])

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
#0.8765328998395648
#0.8723729179726375    result of change hidden layer from 8 to 32
#0.9065247187071686    result of add one layer 32 between encoder :)
#0.904416749009233     result add hidden layer 32 to decoder too and add decoder to ruad model with gpu (this is pretty good)
#0.8655186444515994    result first ruad model with gpu and add timestamp column and woindow 10 and seq , adam optimizer
#0.8967381497429511    result first ruad model with gpu and add timestamp column and woindow 10 and remove seq , adam optimizer
#0.3060959342747257    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, adam optimizer
#0.34766214604908874   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer
#0.36209899126422795   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 +3 (i forgot to reset)
#0.907213292243243     result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 , add , h,c
#0.9098181020482538    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 6 , add , h,c
#0.9069957536027086    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq, SGD optimizer , epoch 6 , add , h,c --- moshtrak
#0.9345063164717007    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1351, with time
#0.6694775694129658    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1351 , with time
#0.6529441138631129    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 6 ,len(data)=1351, with time
#0.9342669032465992    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=1350, (remove time)

#### new ones
#1.0                  result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=353 (time) - create_data_my_model_seprate_anomaly_0
#0.9999983344614581   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=353 (time) - create_data_my_model_add_label_2_lr_0.1
#0.3703446862316149   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=353 (time) - create_data_my_model_add_label_2_lr_0.03
#0.9711356428         result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c ,len(data)=353 (time) - create_data_my_model_add_label_2_lr_0.01 ## 0.0288643572 error hastesh


